### Filting cases

In [ ]:
#case_filter.py
import pandas as pd
import numpy as np

reader=pd.read_csv('/scratch/fs1520/case_tfidf.csv',chunksize=100000)

def ran_drop(source,outpath):
        a=pd.DataFrame()
        counter=0
        for i in source:
                tp=i.sample(frac=0.5)
                if counter==0:
                        a=tp
                else:
                        a=pd.concat([a,tp],ignore_index=True)
                counter+=1

        #a.to_csv('/scratch/fs1520/case_tfidf.csv')
        return(a)

rec=ran_drop(reader,'aa.csv')
rec.to_csv('/scratch/fs1520/case_tfidf.csv')

### Merge case, nn embedding and zip

In [ ]:
import pandas as pd
from sklearn import preprocessing

ca_op_NN=pd.read_csv('/scratch/fs1520/case_nn.csv')
zip_data=pd.read_csv('/scratch/fs1520/zipcode_census.csv')
ca_op_NN=ca_op_NN.drop('Unnamed: 0',axis=1).drop('Unnamed: 0_x',axis=1).drop('Unnamed: 0_y',axis=1).drop('district',axis=1)
zipdata=zip_data.drop('Unnamed: 0',axis=1)
zipdata['EMP']=preprocessing.scale(zipdata['EMP'])
zipdata['QP1']=preprocessing.scale(zipdata['QP1'])
zipdata['AP']=preprocessing.scale(zipdata['AP'])
zipdata['EST']=preprocessing.scale(zipdata['EST'])
zipdata['population']=preprocessing.scale(zipdata['population'])
zip_data.to_csv('/scratch/fs1520/zip_normed.csv')

ca_op_NN=ca_op_NN.fillna(0)
ca_op_NN['zip1']=ca_op_NN['zip1'].apply(lambda x: int(x))
ca_op_NN_mix=pd.merge(ca_op_NN,zipdata,how='inner',left_on='zip1',right_on='zipcode')
ca_op_NN_mix=ca_op_NN_mix.drop('zipcode',axis=1).drop('zip1',axis=1).drop('zip',axis=1)

# ca_op_NN_mix['EMP']=preprocessing.scale(ca_op_NN_mix['EMP'])
# ca_op_NN_mix['QP1']=preprocessing.scale(ca_op_NN_mix['QP1'])
# ca_op_NN_mix['AP']=preprocessing.scale(ca_op_NN_mix['AP'])
# ca_op_NN_mix['EST']=preprocessing.scale(ca_op_NN_mix['EST'])
# ca_op_NN_mix['population']=preprocessing.scale(ca_op_NN_mix['population'])

ca_op_NN_mix.to_csv('/scratch/fs1520/ca_nn_zip.csv')

### Merge case, tfidf embedding and zip

In [ ]:
import pandas as pd
from sklearn import preprocessing

ca_op_tfidf=pd.read_csv('/scratch/fs1520/case_tfidf.csv')
zip_data=pd.read_csv('/scratch/fs1520/zipcode_census.csv')
ca_op_tfidf=ca_op_tfidf.drop('Unnamed: 0',axis=1).drop('Unnamed: 0.1',axis=1).drop('district',axis=1)
zipdata=zip_data.drop('Unnamed: 0',axis=1)
zipdata['EMP']=preprocessing.scale(zipdata['EMP'])
zipdata['QP1']=preprocessing.scale(zipdata['QP1'])
zipdata['AP']=preprocessing.scale(zipdata['AP'])
zipdata['EST']=preprocessing.scale(zipdata['EST'])
zipdata['population']=preprocessing.scale(zipdata['population'])
zip_data.to_csv('/scratch/fs1520/zip_normed.csv')

ca_op_tfidf=ca_op_tfidf.fillna(0)
ca_op_tfidf['zip1']=ca_op_tfidf['zip1'].apply(lambda x: int(x))
ca_op_tfidf_mix=pd.merge(ca_op_tfidf,zipdata,how='inner',left_on='zip1',right_on='zipcode')
ca_op_tfidf_mix=ca_op_tfidf_mix.drop('zipcode',axis=1).drop('zip1',axis=1).drop('zip',axis=1)

# ca_op_tfidf_mix['EMP']=preprocessing.scale(ca_op_tfidf_mix['EMP'])
# ca_op_tfidf_mix['QP1']=preprocessing.scale(ca_op_tfidf_mix['QP1'])
# ca_op_tfidf_mix['AP']=preprocessing.scale(ca_op_tfidf_mix['AP'])
# ca_op_tfidf_mix['EST']=preprocessing.scale(ca_op_tfidf_mix['EST'])
# ca_op_tfidf_mix['population']=preprocessing.scale(ca_op_tfidf_mix['population'])

ca_op_tfidf_mix.to_csv('/scratch/fs1520/ca_tfidf_zip.csv')

### Merge case and nn embedding

In [ ]:
import pandas as pd
def inner_merger(the_cases,the_opinions,lk_1,lk_2,rk_1,rk_2,out_name):
    ca_n_op=pd.merge(the_cases,the_opinions,how='inner',left_on=[lk_1,lk_2],right_on=[rk_1,rk_2])
    ca_n_op.to_csv(out_name)
    return(ca_n_op)

cases=pd.read_csv('/scratch/fs1520/case_with_year.csv')
cases['judge']=cases['judge'].apply(lambda x:str(x).lower())
embedded=pd.read_csv('/scratch/fs1520/embedding.csv')

embedded["judge"] = embedded["judge_dist"].map(lambda x:x.split(' ')[0])
embedded["dist"] = embedded["judge_dist"].map(lambda x:x.split(' ')[1])
embedded=embedded.drop('judge_dist',axis=1)


ca_op_NN=inner_merger(cases,embedded,'district','judge','dist','judge','/scratch/fs1520/case_nn.csv')

### Merge case and tfidf embedding

In [ ]:
#tfidf merger
import pandas as pd
def inner_merger(the_cases,the_opinions,lk_1,lk_2,rk_1,rk_2,out_name):
    ca_n_op=pd.merge(the_cases,the_opinions,how='inner',left_on=[lk_1,lk_2],right_on=[rk_1,rk_2])
    ca_n_op.to_csv(out_name)
    return(ca_n_op)

cases=pd.read_csv('/scratch/fs1520/case_with_year.csv')
cases['judge']=cases['judge'].apply(lambda x:str(x).lower())

tfidfed=pd.read_csv('/scratch/fs1520/opinions_tfidf_feature.csv',delimiter=',')
tfidfed["judge"] = tfidfed["judge_dist"].map(lambda x:x.split(' ')[0])
tfidfed["dist"] = tfidfed["judge_dist"].map(lambda x:x.split(' ')[1])
tfidfed=tfidfed.drop('judge_dist',axis=1)

ca_op_tfidf=inner_merger(cases,tfidfed,'district','judge','dist','judge','/scratch/fs1520/case_tfidf.csv')

### Merge cases and judge biography

In [ ]:
reader_all=[]
reader_all.append(pd.read_stata('/data/Dropbox/Apps/Data/Bankrupty_Courts/Data/Clean/All_data_Part_1.dta',chunksize=10000))
reader_all.append(pd.read_stata('/data/Dropbox/Apps/Data/Bankrupty_Courts/Data/Clean/All_data_Part_2.dta',chunksize=10000))
reader_all.append(pd.read_stata('/data/Dropbox/Apps/Data/Bankrupty_Courts/Data/Clean/All_data_Part_3.dta',chunksize=10000))


#test data
for i in reader_all[0]:
	print(i.columns)
	break


def judge_case_merger(df1,datasource2,df1_judge,df2_judge,df_inherit=False):
#df1: small dataset to be merged datasource2:big one df1_judge:clean_judge df2_judge:judge
	df_counter=0
	base_frame=pd.DataFrame()
	for df2 in datasource2:
		df2['judge']=df2['judge'].apply(lambda x:str(x).lower())
		temp_merged=pd.merge(df2,small_data,how='inner',left_on='judge',right_on='clean_judge')
		if df_counter==0:
			base_frame=temp_merged
		else:
			base_frame=pd.concat([base_frame,temp_merged])
		df_counter+=1
	return(base_frame)

merged=[]
merged.append(judge_case_merger(small_data,reader_all[0],'judge','clean_judge'))
merged.append(judge_case_merger(small_data,reader_all[1],'judge','clean_judge'))
merged.append(judge_case_merger(small_data,reader_all[2],'judge','clean_judge'))


df2=pd.read_csv('judge_case_unique.csv')
df2['0']=df2['0'].apply(lambda x:str(x).lower())
df2['clean_judge'].str.split(' ',expand=True)

### Feature engineering for judges and courts

In [ ]:
def judge_unique(datasource2,df2_judge,df_inherit=False): ###unique judges
	judge_all=set([])
	for df2 in datasource2:
		judge_all=judge_all.union(df2[df2_judge].unique())
	
	return(judge_all)

def get_all_unique_judge(source):
	judge_all_true=set([])
	for i in source:
		judge_all_true=judge_all_true.union(judge_unique(i,'judge',df_inherit=False))
	pd.DataFrame(list(judge_all_true)).to_csv('/data/WorkData/judge_case_unique.csv')
	return(judge_all_true)

def court_unique(datasource2,df2_court,df2_judge,df_inherit=False): ###unique court
	judge_all=set([])
	for df2 in datasource2:
		judge_all=judge_all.union(df2[[df2_court,df2_judge]].apply(tuple,axis=1).unique())
	judge_pd=pd.DataFrame(judge_all,columns=['district','judge'])
	return(judge_all)

def get_all_unique_court(source):
	court_all_true=set([])
	for i in source:
		court_all_true=court_all_true.union(court_unique(i,'district','judge',df_inherit=False))
	pd.DataFrame(list(court_all_true)).to_csv('/data/WorkData/court_judge_case_unique.csv')
	return(court_all_true)



### Merge the cases.dta for the whole country, sampling and cleaning

In [ ]:
reader_all=[]
reader_all.append(pd.read_stata('/data/Dropbox/Apps/Data/Bankrupty_Courts/Data/Clean/All_data_Part_1.dta',chunksize=10000))
reader_all.append(pd.read_stata('/data/Dropbox/Apps/Data/Bankrupty_Courts/Data/Clean/All_data_Part_2.dta',chunksize=10000))
reader_all.append(pd.read_stata('/data/Dropbox/Apps/Data/Bankrupty_Courts/Data/Clean/All_data_Part_3.dta',chunksize=10000))

def fill_disp(datasource,df_inherit=False): ###unique court
	df_counter=0
	case_disp=pd.DataFrame()
	for df in datasource:
		df['disp']=None
		df['the_year']=0
		df.loc[df.loc[:,'dismiss_date']!='','disp']=0
		df.loc[df.loc[:,'discharge_date']!='','disp']=1
		df=df.dropna(subset=['disp'])
		df['close_date']=df['close_date'].apply(lambda x:str(x).split('/')[-1])
		df['close_date']=df['close_date'].apply(lambda x: None if x=='nan' or x=='' else x)
		df['file_date']=df['file_date'].apply(lambda x:str(x).split('/')[-1])
		df['file_date']=df['file_date'].apply(lambda x: None if x=='nan' or x=='' else x)
		df=df.dropna(subset=['close_date'])
		df=df.dropna(subset=['file_date'])
		df['close_date']=df['close_date'].apply(lambda x:int(x)*365+np.random.randint(365))
		df['file_date']=df['file_date'].apply(lambda x:int(x)*365)
		df['time_dif']=df['close_date']-df['file_date']
		df_tp=df[['district','judge','time_dif']] #zip #
		if df_counter==0:
			case_disp=df_tp
		else:
			case_disp=pd.concat([case_disp, df_tp], ignore_index=True)
		df_counter+=1
	return(case_disp)

def merge_all_case(source):
	source_counter=0
	case_all=pd.DataFrame()
	for i in source:
		if source_counter==0:
			case_all=fill_disp(i)
		else:
			case_all=pd.concat([case_all, fill_disp(i)], ignore_index=True)
		source_counter+=1
	case_all.to_csv('/data/WorkData/all_cases_merged_disp.csv')
	return(case_all)

all_cases_new=merge_all_case(reader_all)

df2=pd.DataFrame({'mean_time':all_cases_new.groupby(['district','judge'])['time_dif'].mean()}).reset_index()
df2.to_csv('data/WorkData/cases_timedif.csv')